### **Curating body_inguinal1a.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

ImportError: Numba needs NumPy 1.21 or less

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [ ]:
# Load the AnnData object

In [ ]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cakirb/website_objectfix/spatial-skin-atlas/vis_fixed2/fat/WSSKNKCLsp10446623.h5ad')

In [ ]:
# View the AnnData object

In [ ]:
adata

##### **X- expression matrix**

In [ ]:
# View the expression matrix of the anndata object

In [ ]:
adata.X

In [ ]:
#data type of adata.X

In [ ]:
type(adata.X)

In [ ]:
#convert to csr_matrix

In [ ]:
adata.X = csr_matrix(adata.X)

In [ ]:
adata.X

In [ ]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [ ]:
print(adata.X)

In [ ]:
print(adata.raw.X)

##### **Raw counts matrix**

In [ ]:
# If X has normalized counts, check for the raw counts matrix.

In [ ]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [ ]:
#araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp10446623/filtered_feature_bc_matrix.h5')

In [ ]:
# view raw object

In [ ]:
#araw

In [ ]:
# view raw matrix

In [ ]:
araw = adata.raw.to_adata()

In [ ]:
araw.X

In [ ]:
print(araw.X)

In [ ]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [ ]:
#View the var of anndata and raw object

In [ ]:
adata.var

In [ ]:
araw.var

In [ ]:
#Ensembl IDs

In [ ]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [ ]:
gene_info

In [ ]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [ ]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [ ]:
gene_info_genesym_to_ensembl

In [ ]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [ ]:
adata.var['gene_symbols'] = adata.var_names

In [ ]:
araw.var['gene_symbols'] = araw.var_names

In [ ]:
araw.var_names = araw.var['gene_ids']

In [ ]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [ ]:
adata.var

In [ ]:
adata.var['ensembl_id'].isna().sum()

In [ ]:
adata.var_names = adata.var['ensembl_id']

In [ ]:
adata.var

In [ ]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [ ]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [ ]:
adata.var.index= adata.var.index.drop_duplicates()

In [ ]:
adata.var

In [ ]:
araw.var

In [ ]:
# load the approved genes file

In [ ]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [ ]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [ ]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [ ]:
genedict

In [ ]:
len(genedict)

In [ ]:
# Filter out the genes which are not in the approved genes file

In [ ]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [ ]:
len(var_to_keep_adata)

In [ ]:
len(var_to_keep_araw)

In [ ]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [ ]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [ ]:
#  View the var

In [ ]:
adata.var

In [ ]:
araw.var

feature is filtered

In [ ]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [ ]:
add_zero()

In [ ]:
# view var

In [ ]:
adata.var

In [ ]:
list(adata.var['feature_is_filtered'].unique())

In [ ]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [ ]:
True_count

In [ ]:
araw.var

#### **Observations(obs) (Cell metadata)**

In [ ]:
#view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

#### **assay_ontology_term_id**

In [ ]:
# add the assay_ontology_term_id column

In [ ]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **cell_type_ontology_term_id**

In [ ]:
#get the column in adata.obs related. to cell type annotation

In [ ]:
adata.obs.columns

In [ ]:
adata.obsm

In [ ]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [ ]:
c2l_columns

In [ ]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [ ]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [ ]:
adata.obs

In [ ]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [ ]:
# create a dictionary of cell type and ontology term

In [ ]:
# add the cell_type_ontology_term_id column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [ ]:
adata.obs

#### **donor_id**

In [ ]:
#identify the column in adata.obs which provides donor information

In [ ]:
adata.obs.columns

In [ ]:
# add the donor_id column

In [ ]:
adata.obs['donor_id'] = ['body_inguinal_A'] * len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [ ]:
# view unique values of donor_id column

In [ ]:
list(adata.obs['donor_id'].unique())

In [ ]:
#view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

#### **development_stage_ontology_term_id**

In [ ]:
# identify the column in adata which corresponds to age

In [ ]:
# add the development_stage_ontology_term_id column

In [ ]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000149'] * len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [ ]:
# view unique values of development_stage_ontology_term_id column

In [ ]:
list(adata.obs['development_stage_ontology_term_id'].unique())

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **disease_ontology_term_id**

In [ ]:
# Assign normal since all are healthy patients

In [ ]:
# add the disease_ontology_term_id column

In [ ]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [ ]:
#change data type of column

In [ ]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **is_primary_data**

In [ ]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [ ]:
adata.obs

In [ ]:
#change data type of column

In [ ]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [ ]:
# assign organism id 

In [ ]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [ ]:
#change data type of column

In [ ]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **self_reported_ethnicity_ontology_term_id**

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

#### **sex_ontology_term_id**

In [ ]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [ ]:
adata.obs

#### **suspension_type**

In [ ]:
# since visium suspension type is 'na'

In [ ]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **tissue_type**

In [ ]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [ ]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [ ]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0008337'] * len(adata.obs)

In [ ]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [ ]:
list(adata.obs['tissue_ontology_term_id'].unique())

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [ ]:
adata.obsm

In [ ]:
adata.obsm.keys()

#### **uns (Dataset Metadata)**

In [ ]:
adata.uns

In [ ]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the inguinal part of abdomen stained with H&E'

In [ ]:
adata.uns['title'] = 'Visium spatial - body_inguinal_A_a'

In [ ]:
adata.uns['default_embedding'] = 'X_spatial'

In [ ]:
adata.uns.keys()

### **Final checks and adjustments**

In [ ]:
adata

In [ ]:
adata.obs.dtypes

In [ ]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [ ]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
#check the format of expression matrix

In [ ]:
adata.X

In [ ]:
araw.X

In [ ]:
#Copy raw counts to adata.raw

In [ ]:
adata.raw = araw

In [ ]:
del adata.uns['log1p']

In [ ]:
#write the curated object to final_objects folder

In [ ]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/body_inguinal1a.h5ad', compression = 'gzip')